PROBLEM 1

In [4]:
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from PIL import Image
from sklearn.datasets import fetch_openml

minst = fetch_openml('mnist_784', cache=True)


In [70]:
x = minst.data.astype('float32')
y = minst.target.astype('int64')
x['target'] = y
X = []
for i in range(10):
  X.append(x[x['target']==i])
  X[i] = X[i].sample(frac=1).reset_index(drop=True)

In [73]:
# split the data into training and testing
# train ratio is 70%
# test ratio is 15%
# validation ratio is 15%
# transform the data into a torch tensor and normalize it to be between 0 and 1
rowRange = y.value_counts()
train = []
test = []
validation = []
trainY = []
validationY = []
testY = []

def transform(array):
    array[i] = array[i].to_numpy()
    array[i] = torch.Tensor(array[i])
    array[i] = array[i]/255

def getSizes(index):
    trainSize = int(0.7 * len(X[index]))
    validationSize = int(0.15 * len(X[index]))
    testSize = len(X[index]) - trainSize - validationSize
    return trainSize, validationSize, testSize

for i in range(10):
    trainSize, validationSize, testSize = getSizes(i)
    train.append(X[i][:trainSize])
    validation.append(X[i][trainSize:trainSize+validationSize])
    test.append(X[i][trainSize+validationSize:])

    trainY.append(train[i]['target'].to_numpy())
    validationY.append(validation[i]['target'].to_numpy())
    testY.append(test[i]['target'].to_numpy())
    # cleanup
    del train[i]['target']
    del validation[i]['target']
    del test[i]['target']
    transform(train)
    transform(test)
    transform(validation)

    
    

Problem 2:

In [157]:
# Binary classification via KNN for 2 and 3 combination
# 2 is the positive class and 3 is the negative class
# implement the KNN algorithm
# use the Euclidean distance and the Manhattan distance
# use the KNN algorithm to classify the test data

# create data and label concatenation 2 and 3

data = torch.cat((train[2], train[3]), 0)
dataLabel = np.concatenate((trainY[2], trainY[3]), 0)
validation = torch.cat((validation[2], validation[3]), 0)
validationLabel = np.concatenate((validationY[2], validationY[3]), 0)


In [142]:
def norm(x,y, type):
  if(type=="L1"):
    return torch.norm(x-y, p=1, dim=1)
  elif(type=="L2"):
    return torch.norm(x-y, p=2, dim=1)

def knn(data, dataLabel, validation, validationLabel, k, distance_type):
  hitCount = 0
  for target in range(len(validation)):
    result = norm(data, validation[target], distance_type)
    # get the k smallest distances
    knn = result.topk(k, largest=False)
    count = Counter(dataLabel[knn.indices])
    hit = count.most_common(1)[0][0]
    if(hit == validationLabel[target]):
      hitCount += 1
  return round(hitCount/len(validation)*100, 2)

print("Problem 2 ANSWER (a)")
pred = knn(data, dataLabel, validation, validationLabel, 5, "L1")
pred

Problem 2 ANSWER (a)


99.34

#2 b) What are the hyperparameters you can tune?

In [ ]:
# b) What are the hyperparameters you can tune?
# k, distance type

c) Try at least two other options for each hyperparameter. Report the performance for
each option.

In [83]:
# c) Try at least two other options for each hyperparameter. Report the performance for
# each option.
print("Problem 2 ANSWER (c)")
for k in range(3,7,2):
    predL1 = knn(data, dataLabel, validation, validationLabel, k, "L1")
    predL2 = knn(data, dataLabel, validation, validationLabel, k, "L2")
    print("K ", k)
    print("L1 (Manhatten Distance): ", predL1)
    print("L2 (Euclidean Distance): ", predL2)


Problem2 - (c)
K  3
L1  99.48
L2  99.67
K  5
L1  99.34
L2  99.43


d) You can try more options if you want. What is the final test accuracy?

In [109]:
# d) You can try more options if you want. What is the final test accuracy?
predL2 = knn(data, dataLabel, validation, validationLabel, 3, "L2")
print("K ", 3)
print("L2 (Euclidean Distance): ", predL2)

K  3
L2 (Euclidean Distance):  99.48


Problem #3: Multiclass Classification via k-NN [15pts]

a) Make a k-NN (k=5) and its training/validation/evaluation code to perform multiclass
classification over all digits.

In [112]:
# a) Make a k-NN (k=5) and its training/validation/evaluation code to perform multiclass
# classification over all digits.
data = torch.cat((train[0], train[1]), 0)
dataLabel = np.concatenate((trainY[0], trainY[1]), 0)
validation = torch.cat((validation[0], validation[1]), 0)
validationLabel = np.concatenate((validationY[0], validationY[1]), 0)

In [121]:
print("Problem 3 ANSWER (a)")
Pred = knn(data, dataLabel, validation, validationLabel, 5, "L2")
print("Multiclass KNN Accuracy: ", Pred)

Problem 3 ANSWER (a)
Multiclass KNN Accuracy:  99.2


b) What are the hyperparameters you can tune?

In [ ]:
# b) What are the hyperparameters you can tune?
# k, distance type

c) Try at least two other options for each hyperparameter. Report the performance for
each option.

In [122]:
# c) Try at least two other options for each hyperparameter. Report the performance for
# each option.
print("Problem 3 ANSWER (c)")
for k in range(3,7,2):
    predL1 = knn(data, dataLabel, validation, validationLabel, k, "L1")
    predL2 = knn(data, dataLabel, validation, validationLabel, k, "L2")
    print("K ", k)
    print("L1 (Manhatten Distance): ", predL1)
    print("L2 (Euclidean Distance): ", predL2)

Problem 3 ANSWER (c)
K  3
L1 (Manhatten Distance):  99.06
L2 (Euclidean Distance):  99.15
K  5
L1 (Manhatten Distance):  99.15
L2 (Euclidean Distance):  99.2


In [127]:
print("Problem 3 ANSWER (d)")
# d) You can try more options if you want. What is the final test accuracy?
predL2 = knn(data, dataLabel, validation, validationLabel, 3, "L2")
print("K ", 3)
print("Multiclass L2 (Euclidean Distance) (AUC): ", predL2)

Problem 3 ANSWER (d)
K  3
Multiclass L2 (Euclidean Distance) (AUC):  99.15


In [6]:
import torch.nn as nn
import torch.optim as optim

In [7]:
class HingeLoss(nn.Module):
    def __init__(self):
        super(HingeLoss, self).__init__()

    def forward(self, y, y_pred):
        return torch.mean(torch.clamp(1 - y * y_pred, min=0))

In [75]:
# split the data into training and testing
# train ratio is 70%
# test ratio is 15%
# validation ratio is 15%
# prepare the data for training and testing the SVM
# use only torch
X = torch.cat((train[2],train[3]), 0)
Y = torch.Tensor(np.concatenate((trainY[2], trainY[3]), axis=0))
Y[Y==2] = -1
Y[Y==3] = 1
val = torch.cat((validation[2],validation[3]), 0)
val_label = np.concatenate((validationY[2], validationY[3]), axis=0)

In [85]:
# a) Make a SVM classifier and its training/validation/evaluation code to perform binary
# classification between digit 2 and 3.
# b) Train for 10 epochs with batch size 64 (1 epoch is equal to one entire passing of the
# train set; i.e., the entire train set is used for training once).
# c) What is the final training accuracy and validation accuracy?

# ACCURACY
def accuracy(X, Y, model):
  hitCount = 0
  for i in range(len(X)):
    y_pred = model(X[i])
    if(y_pred*Y[i]>0):
      hitCount += 1
  return round(hitCount/len(X)*100, 2)

def SVM(X, Y, val, val_label, learning_rate, epoch, batch_size):
  model = nn.Linear(28*28, 1)
  optimizer = optim.SGD(model.parameters(), lr=learning_rate)
  criterion = HingeLoss()
  for i in range(epoch):
    for j in range(0, len(X), batch_size):
      optimizer.zero_grad()
      y_pred = model(X[j:j+batch_size])
      loss = criterion(Y[j:j+batch_size], y_pred)
      loss.backward()
      optimizer.step()
  print("Training Accuracy: ", accuracy(X, Y, model))
  print("Validation Accuracy: ", accuracy(val, val_label, model))
print("Problem4 - (b)")
SVM(X, Y, val, val_label, 0.1, 10, 64)

Problem4 - (b)
Training Accuracy:  65.92
Validation Accuracy:  83.34


In [87]:
# c) How should we choose the number of iterations to achieve good generalization? Train
# until you think the model has achieved good generalization
# d) What are the hyperparameters you can tune?
# e) Try at least two other options for each hyperparameter. Report the performance for
# each option.
# f) You can try more options if you want. What is the final test accuracy?
# we should choose the number of iterations to achieve good generalization by looking at the validation accuracy and training accuracy
print("Problem 4 ANSWER (C)")
print("train opt 1")
SVM(X, Y, val, val_label, 0.1, 10, 64)
print("train opt 2")
SVM(X, Y, val, val_label, 0.1, 20, 64)
print("train opt 3")
SVM(X, Y, val, val_label, 0.1, 30, 64)
print("problem 4 ANSWER (D)")
# d) What are the hyperparameters you can tune?
# learning rate, epoch, batch size
print("problem 4 ANSWER (E)")
print("learning rate opt 0.01, epoch 10, batch size 64")
SVM(X, Y, val, val_label, 0.01, 10, 64)
print("learning rate opt 0.01, epoch 20, batch size 64")
SVM(X, Y, val, val_label, 0.1, 10, 32)
print("learning rate opt 0.01, epoch 30, batch size 64")
SVM(X, Y, val, val_label, 0.1, 10, 128)
print("learning rate opt 0.1, epoch 10, batch size 32")
SVM(X, Y, val, val_label, 0.1, 10, 32)

Problem 4 ANSWER (C)
train opt 1
Training Accuracy:  64.86
Validation Accuracy:  84.33
train opt 2
Training Accuracy:  66.78
Validation Accuracy:  81.74
train opt 3
Training Accuracy:  68.73
Validation Accuracy:  80.46
problem 4 ANSWER (D)
problem 4 ANSWER (E)
learning rate opt 0.01, epoch 10, batch size 64
Training Accuracy:  85.19
Validation Accuracy:  63.47
learning rate opt 0.01, epoch 20, batch size 64
Training Accuracy:  66.04
Validation Accuracy:  82.63
learning rate opt 0.01, epoch 30, batch size 64
Training Accuracy:  74.76
Validation Accuracy:  73.57
learning rate opt 0.1, epoch 10, batch size 32
Training Accuracy:  66.24
Validation Accuracy:  82.68


In [187]:
# Problem #4: Binary Classification via soft-margin SVM [15pts]
# had a problem running this